In [ ]:
import requests, matplotlib.pyplot as plt, plotly.express as px, pandas as pd, IPython.core.display as ip, matplotlib.dates as mdates
from datetime import datetime

mostRecentSurvey = '2020'
#from epicollect5 api details
surveyURIs = {'2019':'https://five.epicollect.net/api/export/entries/salmon-survey-2019?form_ref=397fba6ecc674b74836efc190840c42d_5d6f454667a28',
              '2020':'https://five.epicollect.net/api/export/entries/salmon-survey-2020?form_ref=f550ab6c4dab44f49bcc33b7c1904be9_5d6f454667a28',
              '2021':'https://five.epicollect.net/api/export/entries/salmon-survey-2021?form_ref=ad5ffedf0a3246a18934e6ec36ed9569_5d6f454667a28'}
salmonURIs = {'2019':'https://five.epicollect.net/api/export/entries/salmon-survey-2019?form_ref=397fba6ecc674b74836efc190840c42d_5d6f509867795',
              '2020':'https://five.epicollect.net/api/export/entries/salmon-survey-2020?form_ref=f550ab6c4dab44f49bcc33b7c1904be9_5d6f509867795',
              '2021':'https://five.epicollect.net/api/export/entries/salmon-survey-2021?form_ref=ad5ffedf0a3246a18934e6ec36ed9569_5d6f509867795'}

#def getEntries(uri):
#    request = requests.get(url = uri)
#    df = pd.DataFrame(pd.json_normalize(request.json()['data']['entries']))
#    while request.json()['links']['next'] is not None:
#        nextURL = request.json()['links']['next']
#        request = requests.get(url = nextURL)
#        df = df.append(pd.json_normalize(request.json()['data']['entries']))
#    return df

def getPage(uri):
    request = requests.get(url = uri)
    #df = pd.DataFrame(pd.json_normalize(request.json()['data']['entries']))
    if request.json()['links']['next'] is not None:
        getEntries(request.json()['links']['next'])
        return df.append(pd.json_normalize(request.json()['data']['entries']))
    else:
        return

#def addNextPageResults(request, df):

#may be good to rename to something like filterDfFor
#should salmonDf be an object with filterSalmonDf as a method?
def filterSalmonDf(salmonDf, *filters):
    for filter in filters:
        field, value = filter
        salmonDf = salmonDf.loc[(salmonDf[field] == value)]
    return salmonDf
    
#this function sounds like it's all done in one go when it really gets applied to each survey    
def addStats(survey, salmonDf):
    ###maybe make nested function
    surveyUUID = survey['ec5_uuid']
    #is it really today? no. it's the current date
    todaySalmonDf = salmonDf[salmonDf['ec5_parent_uuid'] == surveyUUID]
    ###
    # feed in tuples or something to avoid so much boilerplate
    deadChum = filterSalmonDf(todaySalmonDf, ('12_Species', 'Chum'), ('16_Status', 'Dead')).shape[0]
    deadCoho = filterSalmonDf(todaySalmonDf, ('12_Species', 'Coho'), ('16_Status', 'Dead')).shape[0]
    liveChum = filterSalmonDf(todaySalmonDf, ('12_Species', 'Chum'), ('16_Status', 'Live')).shape[0]
    liveCoho = filterSalmonDf(todaySalmonDf, ('12_Species', 'Coho'), ('16_Status', 'Live')).shape[0]
    remnant = filterSalmonDf(todaySalmonDf, ('16_Status', 'Remnant')).shape[0]
    cutthroat = filterSalmonDf(todaySalmonDf, ('12_Species', 'Cutthroat')).shape[0]
    totalDead = todaySalmonDf.loc[(todaySalmonDf['16_Status'] == 'Dead')].shape[0]
    totalLiveSalmon = liveChum + liveCoho
    total = (todaySalmonDf.shape[0])
    #maybe can feed the inputs in a less terrible way
    return pd.Series({'Survey Date': datetime.strptime(survey['1_Survey_Date'], '%m/%d/%Y').date(), 'Dead Chum': deadChum, 'Dead Coho': deadCoho, 'Live Chum': liveChum, 'Live Coho': liveCoho, 'Total Dead': totalDead, 'Total Live Salmon': totalLiveSalmon, 'Total': total, 'Remnant': remnant, 'Cutthroat': cutthroat})

def addTotalsColumns(statsDf):
    kwargs = {'Dead to Date' : statsDf['Total Dead'].cumsum(), 'Dead Chum to Date' : statsDf['Dead Chum'].cumsum(), 'Dead Coho to Date' : statsDf['Dead Coho'].cumsum()}
    return statsDf.assign(**kwargs)

salmonDf = getEntries(salmonURIs.get(mostRecentSurvey))
surveyDf = getEntries(surveyURIs.get(mostRecentSurvey))
statsDf = surveyDf.apply(addStats, axis=1, args=[salmonDf]).groupby(['Survey Date']).sum().reset_index()
columnOrder =  ['Survey Date', 'Total', 'Total Live Salmon', 'Total Dead', 'Live Chum', 'Live Coho', 'Dead Chum', 'Dead Coho', 'Remnant', 'Cutthroat']
statsDf = statsDf.sort_values(by=['Survey Date'])[columnOrder]
#feels like this should be a class method
statsDf = addTotalsColumns(statsDf)

In [ ]:
def displayTotalsBySurveyDate(statsDf):
    display(ip.HTML(statsDf[['Survey Date', 'Total', 'Total Live Salmon', 'Total Dead', 'Live Chum', 'Live Coho', 'Dead Chum', 'Dead Coho', 'Remnant', 'Cutthroat']].to_html()))

def displayYearToDateTotals(statsDf):
    max = (statsDf['Dead to Date']+statsDf['Total Live Salmon']).max()
    idxmax = (statsDf['Dead to Date']+statsDf['Total Live Salmon']).idxmax()
    print('Yearly salmon total is:', int(max))
    print('Yearly total of Chum is:', int((statsDf['Dead Chum to Date']+statsDf['Live Chum']).max()))
    print('Yearly total of Coho is:', int((statsDf['Dead Coho to Date']+statsDf['Live Coho']).max()))
    print('Calculated based on date:', statsDf.iloc[idxmax]['Survey Date'])
    print('Based on total dead count:', int(statsDf.iloc[idxmax]['Dead to Date']), '|', 'plus total live count:', int(statsDf.iloc[idxmax]['Total Live Salmon']))
    
displayTotalsBySurveyDate(statsDf)
displayYearToDateTotals(statsDf)

In [ ]:
def displayCountPlot(statsDf):    
    plot = statsDf.plot(ylabel = 'Count', title = 'Fish Count', rot=45, y=['Total', 'Total Dead', 'Live Chum', 'Dead Chum', 'Live Coho', 'Dead Coho'], x='Survey Date')
displayCountPlot(statsDf)

In [ ]:
def displayHeatMap(salmonDf):    
    fig = px.density_mapbox(salmonDf, lat='15_Location.latitude', lon='15_Location.longitude', radius=15,
                        center=dict(lat=47.71157, lon=-122.3759), zoom=15, hover_name = 'title', hover_data = ['15_Location.accuracy'],
                        mapbox_style='stamen-terrain', title='Fish Heat Map')
    fig.layout.coloraxis.showscale = False
    fig.show()
displayHeatMap(salmonDf)

In [ ]:
def displaySpawningPercentagesChart(salmonDf):
    pie, ax1 = plt.subplots()
    deadChumDf = filterSalmonDf(salmonDf, ('12_Species', 'Chum'), ('16_Status', 'Dead'))
    totalDeadChum = deadChumDf.shape[0]
    totalSpawnedChum = filterSalmonDf(deadChumDf, ('24_Spawned', 'Spawned')).shape[0]
    totalPartialSpawnedChum = filterSalmonDf(deadChumDf, ('24_Spawned', 'Partially spawned')).shape[0]
    totalUnspawnedChum = filterSalmonDf(deadChumDf, ('24_Spawned', 'Unspawned')).shape[0]
    totalUnknownSpawnedChum = filterSalmonDf(deadChumDf, ('24_Spawned', 'Unknown')).shape[0]
    spawnPercentages = [totalSpawnedChum/totalDeadChum*100, totalPartialSpawnedChum/totalDeadChum*100, totalUnspawnedChum/totalDeadChum*100, totalUnknownSpawnedChum/totalDeadChum*100]
    ax1.pie(spawnPercentages, labels=['Spawned', 'Partially Spawned', 'Unspawned', 'Unknown'], autopct='%1.1f%%')
    plt.title('Chum Spawn Percentage')
    plt.show()
displaySpawningPercentagesChart(salmonDf)

In [ ]:
def getMultiyearStats():
    surveyDfs = {};
    salmonDfs = {};
    statsDfs = {};
    for year in surveyURIs:
        surveyDfs[year] = getEntries(surveyURIs[year])
    for year in salmonURIs:
        salmonDfs[year] = getEntries(salmonURIs[year])
    for year in surveyDfs:
        if not salmonDfs[year].empty:
            statsDfs[year] = surveyDfs[year].apply(addStats, axis=1, args=[salmonDfs[year]]).groupby(['Survey Date']).sum().reset_index()
            statsDfs[year]['Survey Date'] = statsDfs[year]['Survey Date'].apply(lambda x: datetime.strptime(x.strftime("%m-%d"), "%m-%d"))
getMultiyearStats()

In [ ]:
def displayYearOverYearCount(surveyURIs, salmonURIs):

#plot each series
    fig, ax = plt.subplots()
    plt.plot('Survey Date', 'Total', data=statsDfs['2019'], label="2019")
    plt.plot('Survey Date', 'Total', data=statsDfs['2020'], label="2020")
    plt.title('Count by time of year')
    plt.ylabel('Count')
    plt.xlabel('Survey Date')
    plt.xticks(rotation = 45)
    plt.legend()
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%m-%d'))

displayYearOverYearCount(surveyURIs, salmonURIs)